Vamos a utilizar el mismo procedimiento usado con los _tweets_ en otros textos

In [12]:
import string
import re
from itertools import product

import pandas as pd
 
import spacy
from spacy.lang.es.stop_words import STOP_WORDS

**Limpieza**

In [2]:
def camel_case(example):      
    if  any(x in example for x  in string.punctuation)==True:
        return False
    else:
        if any(list(map(str.isupper, example[1:-1]))) and not all(list(map(str.isupper, example[1:-1]))):
            return True
        else:
            return False

def camel_case_split(word):
    idx = list(map(str.isupper, word))
    case_change = [0]
    for (i, (x, y)) in enumerate(zip(idx, idx[1:])):
        if x and not y:  
            case_change.append(i)
        elif not x and y:  
            case_change.append(i+1)
    case_change.append(len(word))
    return [word[x:y] for x, y in zip(case_change, case_change[1:]) if x < y]

In [18]:
nlp = spacy.load("es_core_news_lg")
 
def clean_tweets(tweet):
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
 
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
 
    # remove hyperlinks
    tweet = re.sub('http\S*','',tweet)
    
    # remove users
    tweet = re.sub(r'@[a-zA-Z0-9]+', '', tweet)
    
    # split camel case
    camel_case_words = [word[1:] for word in re.findall(r'#[a-zA-Z]+', tweet) if camel_case(word[1:])]
    splited_words = [' '.join(camel_case_split(word[1:])) for word in re.findall(r'#[a-zA-Z]+', tweet) if camel_case(word[1:])]
    for camel, splited in zip(camel_case_words, splited_words):
        tweet = tweet.replace(camel, splited)

    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    
    # remove single letters
    tweet = re.sub(r"\b[a-zA-Z]\b", "", tweet)
 
    # remove multiple spaces
    tweet = re.sub(r"\s+", " ", tweet)

    # tokenize tweets
    tweet_tokens = nlp(tweet)
 
    tweets_clean = [word.lemma_ for word in tweet_tokens if not word.is_stop and word.pos_ != 'PUNCT']
 
    return ' '.join(tweets_clean).lower().strip()

**Clasificación**

In [19]:
palabras = pd.read_feather('../data/diccionario_palabras_ods')

words_list = palabras[['ODS.1', 'PALABRAS']].dropna().groupby('ODS.1')['PALABRAS'].apply(list)
words_list_nlp = [nlp(' '.join(words).lower()) for words in words_list]

## [Objetivo 1: Fin de la pobreza](https://www.undp.org/content/undp/es/home/sustainable-development-goals/goal-1-no-poverty.html)

In [20]:
ods_1 = """
Erradicar la pobreza en todas sus formas sigue siendo uno de los principales desafíos que enfrenta la humanidad. Si bien la cantidad de personas que viven en la extrema pobreza disminuyó en más de la mitad entre 1990 y 2015, aún demasiadas luchan por satisfacer las necesidades más básicas.

A nivel mundial, alrededor de 736 millones de personas aún viven con menos de US$1,90 al día (2015) y muchos carecen de acceso a alimentos, agua potable y saneamiento adecuados. El crecimiento económico acelerado de países como China e India ha sacado a millones de personas de la pobreza, pero el progreso ha sido disparejo. La posibilidad de que las mujeres vivan en situación de pobreza es desproporcionadamente alta en relación con los hombres, debido al acceso desigual al trabajo remunerado, la educación y la propiedad.

Los avances también han sido limitados en otras regiones, como Asia Meridional y África subsahariana, donde vive el 80% de la población mundial que se encuentra en condiciones de extrema pobreza. Además, nuevas amenazas que plantean el cambio climático, los conflictos y la inseguridad alimentaria necesitan mayores esfuerzos para sacar a las personas de la pobreza.

Los Objetivos de Desarrollo Sostenible constituyen un compromiso audaz para finalizar lo que comenzamos y terminar con la pobreza en todas sus formas y dimensiones para 2030. Esto requiere enfocarse en los más vulnerables, aumentar el acceso a los recursos y servicios básicos y apoyar a las comunidades afectadas por conflictos y desastres relacionados con el clima.
"""

In [21]:
ods_1 = clean_tweets(ods_1)
ods_1

'erradicar pobreza formar principal desafío enfrentar humanidad cantidad personar vivir extremo pobreza disminuir mitad 1990 2015 demasiar luchar satisfacer necesidad básico nivel mundial 736 millón personar vivir us,90 2015 carecer acceso alimento aguar potable saneamiento adecuar crecimiento económico acelerar país china india sacar millón personar pobreza progresar disparejo posibilidad mujer vivir situación pobreza desproporcionadamente alto relación hombre acceso desigual remunerar educación propiedad avanzar limitar región asia meridional áfrica subsahariano vivir 80% población mundial condicionar extremo pobreza amenazar plantear cambiar climático conflicto inseguridad alimentario necesitar mayor esfuerzo sacar personar pobreza objetivos desarrollo sostenible constituir compromiso audaz finalizar comenzar terminar pobreza formar dimensionar 2030 requerir enfocarse vulnerable aumentar acceso recurso servicio básico apoyar comunidad afectar conflicto desastre relacionar clima'

In [22]:
[nlp(ods_1).similarity(words) for words in words_list_nlp]

[0.779285550590821,
 0.783839994867686,
 0.8118746492257268,
 0.7394701469184014,
 0.7851740340590067,
 0.5802161881880262,
 0.6060942946677514,
 0.8403538764354862,
 0.7201978175467083,
 0.8364762405365745,
 0.6893683361546529,
 0.7483268347304876,
 0.6654156393107648,
 0.5828273059427406,
 0.7041392289893651,
 0.7797578445934781,
 0.5716903765988367]

## [Objetivo 2: Hambre cero](https://www.undp.org/content/undp/es/home/sustainable-development-goals/goal-2-zero-hunger.html)

In [26]:
ods_2 = """
Debido al rápido crecimiento económico y al aumento de la productividad agrícola en las últimas dos décadas, el número de personas desnutridas disminuyó casi a la mitad. Muchos países en desarrollo que sufrían hambrunas están ahora en condiciones de satisfacer las necesidades nutricionales de los más vulnerables. Regiones como Asia Central y Oriental y América Latina y el Caribe han avanzado enormemente en la erradicación del hambre.

Desgraciadamente, el hambre y la desnutrición siguen siendo grandes obstáculos para el desarrollo de muchos países. Se estima que 821 millones de personas sufrían de desnutrición crónica al 2017, a menudo como consecuencia directa de la degradación ambiental, la sequía y la pérdida de biodiversidad. Más de 90 millones de niños menores de cinco años tienen un peso peligrosamente bajo. La desnutrición y la inseguridad alimentaria parece estar incrementándose tanto en casi todas las de regiones de África, como en América del Sur.

Los Objetivos de Desarrollo Sostenible buscan terminar con todas las formas de hambre y desnutrición para 2030 y velar por el acceso de todas las personas, en especial los niños, a una alimentación suficiente y nutritiva durante todo el año. Esta tarea implica promover prácticas agrícolas sostenibles a través del apoyo a los pequeños agricultores y el acceso igualitario a la tierra, la tecnología y los mercados. Además, se requiere el fomento de la cooperación internacional para asegurar la inversión en la infraestructura y la tecnología necesaria para mejorar la productividad agrícola.
"""

In [29]:
ods_2 = clean_tweets(ods_2)
[nlp(ods_2).similarity(words) for words in words_list_nlp]

[0.7909726479841821,
 0.8249554007345031,
 0.8306335472027054,
 0.7355626098158923,
 0.767764406170211,
 0.6200212340964082,
 0.6375891696077722,
 0.848278938063251,
 0.762007688044762,
 0.8221344810797289,
 0.7384925892420381,
 0.7862445061815598,
 0.6943666316202632,
 0.6250229001193901,
 0.7501406307500155,
 0.7573917737189273,
 0.6058019451447479]

## [El Govern pretende cerrar Punta Ballena y la calle de la cerveza](https://www.diariodemallorca.es/mallorca/2020/07/14/govern-pretende-cerrar-punta-ballena/1523350.html)

In [28]:
noticia = """
El Govern balear pretende ordenar el cierre de las dos calles de Mallorca más conflictivas a causa de las aglomeraciones de turísticas incumpliendo las medidas sanitarias que se están produciendo y cuyas imágenes están dando la vuelta al mundo: Punta Ballena en Magaluf y la conocida como calle de la cerveza en la Platja de Palma. El objetivo es lanzar un mensaje de tranquilidad a los países emisores, especialmente a Alemania, respecto a la imagen de las islas como destino seguro, muy deteriorada durante los últimos días a causa de los vídeos que han circulado con fiestas ilegales y comportamientos incívicos.

Los consellers de Turismo y de Administraciones Públicas, Iago Negueruela e Isabel Castro respectivamente, han comparecido hoy para anunciar un "endurecimiento" de las iniciativas destinadas a evitar las vulneraciones de las medidas sanitarias que se han aprobado, y han concretado que éstas se van a centrar en las zonas tipificadas como mas afectadas por el denominado turismo de excesos, como son Platja de Palma, Magaluf y Sant Antoni de Eivissa. Sin embargo, se han negado a avanzar qué alcance va a tener ese "endurecimiento", al señalar que se está terminando de definir y que previsiblemente se hará público durante el día de mañana. Fuentes del sector turístico han reconocido que entre las iniciativas que se está planteando el Govern aparece el cierre de las dos calles antes señaladas.

Los dos representantes del Ejecutivo balear no han ocultado el daño que el comportamiento irresponsable de unos pocos puede hacer tanto a los residentes en las islas como a sus visitantes, de ahí el objetivo de atajarlo.

Este anuncio por parte de Negueruela y Castro se ha producido tras una reunión celebrada hoy con representantes de las patronales hoteleras y de los sindicatos, que ha sido "muy tensa" en algunos momentos por el rechazo de la parte empresarial a algunas de las iniciativas que ha tomado el Gabinete de Francina Armengol, como es la obligatoriedad de llevar siempre la mascarilla en espacios públicos salvo algunas excepciones.
"""

In [30]:
noticia = clean_tweets(noticia)
[nlp(noticia).similarity(words) for words in words_list_nlp]

[0.6433895807139273,
 0.6479185710223566,
 0.707448111692085,
 0.6593374101530624,
 0.6757161633462677,
 0.5312402277991756,
 0.5129986966287136,
 0.7262882707644215,
 0.6436464179578479,
 0.7026063097073116,
 0.6367470984867267,
 0.6718119150224988,
 0.6137344204541554,
 0.5538252924340162,
 0.6264353700737133,
 0.6987110142551988,
 0.5187347770023865]

## [El Mallorca lo tiene un poco más complicado](https://www.diariodemallorca.es/rcd-mallorca/2020/07/14/complicado/1523250.html)

In [31]:
deportes = """
Seis puntos. Esa es la cifra que debe conseguir el Mallorca para apurar sus ya escasas opciones de permanencia. No queda otra. El Alavés, tras su empate ante el Getafe (0-0), marca el corte de la salvación con 36 puntos, igualado con el Celta, cuatro más de los que el equipo de Vicente Moreno tiene en su haber. Complicado que el Mallorca se salve, cierto, pero no es imposible.

La situación clasificatoria ha pasado de color gris a negro tras un fin de semana catastrófico que no solo ha eliminado al Eibar de la terna de candidatos al descenso, sino que incluso ha incluido en la fiesta a un conjunto que parecía desahuciado como el Leganés, empatado a puntos con los bermellones y con el golaverage directo a favor. Pese a ello, no queda tiempo para lamentaciones. Todo pasa por conseguir los seis puntos que restan y esperar un gran favor del resto del rivales. Podrían bastar cuatro, las matemáticas así lo dictan, pero las probabilidades necesarias para que se dé el caso son ínfimas.

Lo que es seguro es que el Alavés, que al igual que el conjunto pepinero tiene el golaverage a favor con los bermellones, no puede ganar ningún partido de los que restan y sumar como máximo un punto, mientras que el Leganés no debe vencer en alguno de los encuentros que le quedan por delante para que el Mallorca, si cumple con su trabajo, sea un año más equipo de Primera División. El cuarto equipo en la brega, el Celta, que cuenta con 36 puntos, es el que tiene a priori un mejor calendario. Ya sea venciendo en casa al Levante, que no se juega nada más allá de objetivos internos, o consiguiendo el triunfo ante el Espanyol, ya descendido, en Cornellà, los de Óscar García tienen a tiro sellar su permanencia en Primera.

Como pronto, el Mallorca tendrá que tumbar al Granada este jueves en el Visit Mallorca Estadi, que no será nada fácil. Y un dato importante: si Celta y Alavés consiguen el mismo resultado que se produzca en Son Moix, sea victoria, empate o derrota, el Mallorca será nuevo equipo de Segunda División.

La misión que tiene el Mallorca adopta ya unas dimensiones titánicas. Demasiadas variables en juego y la dependencia de otros resultados convierten lograr la permanencia en una gesta épica. Todo puede pasar en el fútbol. En apenas dos días se decidirá todo.
"""

In [32]:
deportes = clean_tweets(deportes)
[nlp(deportes).similarity(words) for words in words_list_nlp]

[0.5014104454057435,
 0.5651478353461546,
 0.5693213403681013,
 0.5366688755709058,
 0.5503250379320013,
 0.38963256698520377,
 0.4439108466611069,
 0.5793873929786267,
 0.4787202090149401,
 0.5421675209289147,
 0.4719464155259662,
 0.5290172132787552,
 0.5205932443057127,
 0.43018135484564807,
 0.48265602497236604,
 0.5194781874594344,
 0.41808651240408895]

## [LEYENDAS DEL ROCK 2021: entradas ya a la venta](https://www.rafabasa.com/2020/07/13/leyendas-del-rock-2021-entradas-ya-a-la-venta/)

In [33]:
concierto = """
¡Las entradas para LEYENDAS DEL ROCK 2021 ya están a la venta! Abono 3 días, camping sombra y piscina. Y lo hacemos con una oferta muy especial, un regalo de 10€ en tokens por abono 3 días adquirido (solo en la red de Ticketmaster). Oferta válida hasta el 31 de julio. También puedes adquirir tu abono a todo color en los puntos habituales.

Puedes adquirir aquí tu abono

Después de las reconfirmaciones de STRATOVARIUS, SARATOGA y JINJER, en las próximas semanas iremos desvelando los nombres que ya teníamos confirmados en 2020 y siguen en cartel, así como el anuncio de nuevas bandas de cara a nuestra edición de 2021. Permanece atento y sobre todo, ¡¡permanece muy heavy!!
"""

In [34]:
concierto = clean_tweets(concierto)
[nlp(concierto).similarity(words) for words in words_list_nlp]

[0.2715914235078668,
 0.39471491881475373,
 0.3246180140628708,
 0.39444278506035546,
 0.28493539896253284,
 0.3056374786514312,
 0.3551473116446221,
 0.3780156746859772,
 0.3761442965355766,
 0.29758496859115285,
 0.3742374562828454,
 0.4455249089094482,
 0.44562735335873216,
 0.23151233403490784,
 0.26529118729360046,
 0.21951367590473841,
 0.218246410865622]